In [1]:
from selenium import webdriver
from selenium.webdriver.coㅌmmon.keys import Keys #키 누를 수 있는 명령어 모음

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys #키 누를 수 있는 명령어 모음
import pymysql
import time
import requests
from bs4 import BeautifulSoup
import re
import time

In [67]:
def keyword_search(arr):
    a = arr.split()
    answer_cnt = 0
    for i in a:
        count = 0;
        for j in a:
            j = j.replace('"', '')
            if i in j and len(i) > 1:
                count+=1;
                if answer_cnt < count:
                    answer_cnt = count
                    answers = i
    return answers

In [87]:
# 웹 스크래핑 부분

what_date = input('어느 일자의 기사들을 보시겠습니까 ("0720" 과 같이 입력하여 주십시오.)')

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='goodo011@', db='naver_news', charset='utf8')
cursor = db.cursor()
text = []

cursor.execute('drop table if exists headlines');
cursor.execute('drop table if exists detail_news');

sql = """
create table headlines
	(
	ord_id varchar(30) not null primary key,
    title varchar(300) not null,
    press varchar(100) not null,
    categori varchar(40) not null,
    url varchar(500) not null
    );
"""

cursor.execute(sql)

sql = """
create table detail_news
	(
    ord_id varchar(30) not null,
    reporter varchar(50) not null,
    date varchar(100),
    keyword varchar(20),
    comment_cnt varchar(20)
    );
"""
cursor.execute(sql)

어느 일자의 기사들을 보시겠습니까 ("0720" 과 같이 입력하여 주십시오.)0520


0

In [88]:
browser = webdriver.Chrome()
url = f'https://news.naver.com/main/ranking/popularDay.naver?date=2021{what_date}'

browser.get(url)

soup = BeautifulSoup(browser.page_source, 'lxml')

rankingnews = soup.find_all('div', {'class' : 'rankingnews_box'})

In [89]:
try:
    want_thing = input('정치, 경제, 사회, 생활, 세계, IT 중에 보고싶은 카테고리를 입력하세요.(모든 카테고리를 희망한다면 "모두"를 입력하세요) : ')
    what_press = input('어느 언론사의 기사들을 보시겠습니까. (모든 언론사를 원하신다면 "모두" 라고 입력하십시오)')
    cnt = int(input('각 언론사에서 몇개의 기사들을 보시겠습니까. (1~5의 값을 입력해주세요)'))


    for ranking in rankingnews:
        press = ranking.find('strong', {'class' : 'rankingnews_name'}).text
        ranks = ranking.find_all('li')
        count = 0
        for rank in ranks:
            if what_press not in press and what_press != '모두':
                continue

            count+=1
            if count > cnt:
                break;
            link = rank.find('a' , {'class' : "list_title nclicks('RBP.rnknws')"})
            
            if not link:
                continue
            elif link:
                title = link.text
                link = 'https://news.naver.com'+link['href']            
                ord_id = link.split('&')[3] + '&' + link.split('&')[4]
                url = link
                browser.get(url)
                soup2 = BeautifulSoup(browser.page_source, 'lxml')
                categori = soup2.find('em' , {'class' : 'guide_categorization_item'})

                if categori and want_thing == categori.get_text():
                    categoris = categori.get_text()
                    print(1)
                elif '모두' in want_thing:
                    categoris = categori.get_text()
                    print(2)
                else:
                    continue
                title = title.replace('"', "'")
                print(ord_id, title, press, categoris, link)
                sql = f"""insert into headlines values("{ord_id}", "{title}", "{press}", '{categoris}', '{link}');"""
                cursor.execute(sql)
                
                reporter = soup2.find('div', {'class' : 'journalistcard_summary_name'})
                
                if reporter:
                    reporter = reporter.text
                    
                date = soup2.find('span', {'class' : 't11'}).text
                texts = soup2.find('div', {'id' : 'articleBodyContents'})
                soup2.select('li > strong')
                texts = texts.text
                keyword = keyword_search(texts)
                comment_cnt = browser.find_element_by_css_selector('span.u_cbox_count').text # 전체 댓글
                comment_cnt = browser.find_element_by_css_selector('span.u_cbox_info_txt').text # 현재 댓글
                print(ord_id, reporter, date, keyword, comment_cnt)
                sql = f"""insert into detail_news values("{ord_id}", "{reporter}", "{date}", '{keyword}', '{comment_cnt}');"""
                cursor.execute(sql)
                time.sleep(1)
finally:
    db.commit()
    db.close()

정치, 경제, 사회, 생활, 세계, IT 중에 보고싶은 카테고리를 입력하세요.(모든 카테고리를 희망한다면 "모두"를 입력하세요) : 모두
어느 언론사의 기사들을 보시겠습니까. (모든 언론사를 원하신다면 "모두" 라고 입력하십시오)뉴스
각 언론사에서 몇개의 기사들을 보시겠습니까. (1~5의 값을 입력해주세요)1
2
oid=001&aid=0012403700 대한민국 최고의 전통미인 '미스춘향 진'에 김민설씨 연합뉴스 사회 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=001&aid=0012403700&rankingType=RANKING
oid=001&aid=0012403700 백도인 2021.05.20. 오전 1:09 남원시 1,337
2
oid=079&aid=0003505870 손정민씨 父, 새로운 목격자 등장에 '대답할 가치 없다' 노컷뉴스 사회 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=079&aid=0003505870&rankingType=RANKING
oid=079&aid=0003505870 CBS노컷뉴스 박하얀 기자 2021.05.19. 오후 7:17 했다. 1,405
2
oid=422&aid=0000485924 '집단폭행 살해' 태권도 유단자들 징역형 확정 연합뉴스TV 사회 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=422&aid=0000485924&rankingType=RANKING
oid=422&aid=0000485924 신민우 2021.05.20. 오전 6:12 뉴스 239
2
oid=421&aid=0005361660 '한강 입수남' 정민씨 맞다면…친구 발견 전후 어딨었나 뉴스1 사회 https://news.naver.com/main/ranking/read.naver?m

In [51]:
                reporter = soup2.find('div', {'class' : 'journalistcard_summary_name'})
                if reporter:
                    reporter = reporter.text
                    
                    
                date = soup2.find('span', {'class' : 't11'}).text
                texts = soup2.find('div', {'id' : 'articleBodyContents'})
                soup2.select('li > strong')
                texts = texts.text
                keyword = keyword_search(texts)
                print(ord_id, reporter, date, keyword)
                sql = f"""insert into detail_news values("{ord_id}", "{reporter}", "{date}", '{keyword}');"""
                cursor.execute(sql)
                time.sleep(1)
                

정치, 경제, 사회, 생활, 세계, IT 중에 보고싶은 카테고리를 입력하세요.(모든 카테고리를 희망한다면 "모두"를 입력하세요) : 모두
어느 언론사의 기사들을 보시겠습니까. (모든 언론사를 원하신다면 "모두" 라고 입력하십시오)모두
각 언론사에서 몇개의 기사들을 보시겠습니까. (1~5의 값을 입력해주세요)1
oid=001&aid=0012518139 55∼59세 백신예약 첫날 또 '먹통'…새벽 3시에 80만명 동시접속 연합뉴스 <em class="guide_categorization_item">사회</em> https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=001&aid=0012518139&rankingType=RANKING


DataError: (1406, "Data too long for column 'categori' at row 1")

In [10]:
#categori = soup2.find('em' , {'class' : 'guide_categorization_item'}).text


'사회'

In [12]:
print(ord_id, link, categori, title, press)

oid=607&aid=0000000778 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=607&aid=0000000778&rankingType=RANKING <em class="guide_categorization_item">사회</em> 전두환 장손, '전두환 그룹' 주인 등극...3대 세습 본격화 레이디경향


In [78]:
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='goodo011@', db='naver_news2', charset='utf8')
cursor = db.cursor()

sql = """insert into headlines values('{ord_id', "{title", '{press}', '{categori', 'link');"""
cursor.execute(sql)

1

In [34]:
keyword_search(texts)

TypeError: 'NoneType' object is not callable

'\n\n\n\n\n\t\n\t[레이디경향] 수박.여름하면 떠오르는 수박은 요즘 누구나 즐겨 먹지만, 조선시대에는 귀족의 과일이었다고 한다. 일단 기후가 맞지 않아 생산량이 적은 것은 물론이고 그 당시에 단맛이 나면서 물이 많은 과일 자체가 별로 없어서 매우 비쌌기 때문이다. 기록에 따르면 세종 때는 수박 한 통 가격이 쌀 다섯 말(40㎏)이었다고 한다.수박은 한자로는 서쪽에서 유래된 박과의 과일이라는 뜻으로 ‘서과(西瓜)’라고 한다. 여기서 서쪽은 중국의 서쪽을 의미한다. 우리가 부르는 ‘수박’은 순우리말이다. 동의보감에 “수박은 성질이 차고[寒] 맛은 달면서[甘] 아주 슴슴하며[極淡] 독이 없다”라고 했다. 그 당시 수박은 맛이 달지만 현대보다는 덜 달고 담백한 맛이었음을 알게 하는 대목이다. 지금 우리가 먹는 수박은 품종 개량을 통해 만들어진 것으로 맛이 달고 색도 빨갛다.수박의 효능에 대해서는 “번갈과 더위 독을 없애고 속을 시원하게 하며 기를 내리고 오줌이 잘 나가게 한다. 혈리(血痢)와 입 안이 헌 것을 치료한다”라고 했는데, 이는 몸에 충분한 수분과 미네랄을 잘 공급한다는 뜻이다. 비뇨기계의 순환을 원활하게 한다는 의미도 있다. 대부분의 오이 과(瓜)처럼 수분이 매우 풍부한 것이 특징이며, 이뇨작용을 돕는 ‘시트룰린’ 성분이 많이 함유돼 있다.수분을 제외하고는 대부분 탄수화물(당분)이며 단백질과 식이섬유 및 미네랄도 상대적으로 매우 적게 함유하고 있다. 다만 탈수증에 맞는 수액주사처럼 빠르게 흡수되는 수분과 당분, 양은 적지만 골고루 함유돼 있는 미네랄 등 때문에 앞서 말한 ‘번갈과 더위 독’ 등 더운 여름에 지치고 힘들었을 때는 딱 맞는 음식이라 할 수 있다.수박의 효능 한 가지를 추가하자면 베타카로틴을 비롯한 카로티노이드 계열 성분들이 비교적 많은데, 이들은 인체 내에서 레티놀과 같은 활성형 비타민A로 전환돼 시각기능, 세포 분화, 항산화, 생식능력 등에 도움을 준다.TIP1, 수박과 체질=수박을 먹으면 배가 아프거나 설사를 해서 잘 먹지 못하

In [70]:
keyword_search(texts)

1 [레이디경향]
2 수박은
3 수박은
4 수박은
5 수박은
6 수박은
7 수박은
8 수박
9 수박
10 수박
11 수박
12 수박
13 수박
14 수박
15 수박
16 수박


'수박'

In [74]:
            reporter = soup2.find('div', {'class' : 'journalistcard_summary_name'}).text
            date = soup2.find('span', {'class' : 't11'}).text
            comment_cnt = browser.find_element_by_css_selector('span.u_cbox_info_txt')
            print(comment_cnt.get_attribute(''))
            texts = soup2.find('div', {'id' : 'articleBodyContents'})
            soup2.select('li > strong')
            texts = texts.text
            keyword = keyword_search(texts)
            #sql = f"""insert into detail_news values("{ord_id}", "{reporter}", "{date}", '{comment_cnt}', '{keyword}');"""
            #cursor.execute(sql)
            #time.sleep(2)           

None
1 [레이디경향]
2 수박은
3 수박은
4 수박은
5 수박은
6 수박은
7 수박은
8 수박
9 수박
10 수박
11 수박
12 수박
13 수박
14 수박
15 수박
16 수박


In [79]:
db.commit()
db.close()

12593

In [27]:
rankingnews = soup.find_all('div', {'class' : 'find_all'})
for ranking in rankingnews:
    press = ranking.find('strong', {'class' : 'rankingnews_name'}).text
    print(press)
    ranks = ranking.find_all('li')
    print(ranks)
    for rank in ranks:
        link = rank.find('a' , {'class' : "list_title nclicks('RBP.rnknws')"})[href]
        print(link)

In [ ]:
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='new', charset='utf8')

dbname = db.cursor()

In [55]:
# 웹 스크래핑 부분


db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='naver_news', charset='utf8')
cursor = db.cursor()
print(cursor)
text = []
sql = """
create table headlines
	(
	ord_id varchar(30) not null primary key,
    title varchar(300) not null,
    press varchar(100) not null,
    categori varchar(40) not null,
    url varchar(500) not null
    );
"""

cursor.execute(sql)
sql = """
create table detail_news
	(
    ord_id varchar(30) not null,
    reporter varchar(50),
    date varchar(20),
    keyword varchar(20)
    );
"""
cursor.execute(sql)

In [ ]:
                sql = f"""insert into headlines values("{ord_id}", "{title}", "{press}", '{categori}', '{link}');"""
                #time.sleep(1)
                cursor.execute(sql)

In [23]:
browser = webdriver.Chrome()

In [24]:
url = 'https://news.naver.com/dsddsadjl;asdal;dka;'
browser.get(url)

soup = BeautifulSoup(url, 'lxml')
soup 지금 현재 사이트의 html내용

In [ ]:
import pymysql

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='new', charset='utf8')

dbname = db.cursor()

sql = """
create table headlines
	(
    title varchar(300) not null,
    press varchar(100) not null,
    categori varchar(20) not null,
    url varchar(100) not null
    );
"""
sql = """
create table detail_news
	(
    ord_id varchar(30) not null,
    reporter varchar(20) not null,
    date int unsigned,
    comment int unsigned,
    text longtext
    );
"""

sql = """insert into headlines values(f'{ord_id}', f'{title}', f'{press}', f'{categori}', f'{link}');"""
sql = """insert into detail_news values(f'{ord_id}', f'{reporter}', f'{date}', f'{comment}', f'{text}');"""

dbname.execute(sql)

db.commit()
db.close()

In [60]:
texts

'\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t화이자의 코로나 백신을 맞은 후 근육통, 발열 등 이상반응 사례는 1차 접종보다 2차 접종 때 더 흔하다는 국내 연구 결과가 나왔습니다. 서울아산병원 배성만 감염내과 교수 연구팀은 이 병원에서 화이자 백신 1, 2차 접종을 완료한 265명을 분석해 이같이 밝혔습니다.그 결과 3일 내 이상반응 비율은 1차 접종 때 80.1%, 2차 접종 때 89.1%로, 근육통과 피로, 두통, 오한, 발열 순이었습니다.연구팀은 화이자 백신이 1차 접종보다 2차 접종 후에 이상반응이 더 자주 발생한다고 보고된 해외 연구와 비슷한 맥락이라고 봤습니다.연합뉴스TV 기사문의 및 제보 : 카톡/라인 jebo23 ▶ 네이버에서 연합뉴스TV를 구독하세요 ▶ 연합뉴스TV 생방송 만나보기  ▶ 균형있는 뉴스, 연합뉴스TV 앱 다운받기\n\n'

In [104]:
'231' - '23'

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [6]:
#text=[]
#reporter = soup2.find('div', {'class' : 'journalistcard_summary_name'})
#date = soup2.find('span', {'class' : 't11'}).text
comment_cnt = soup2.find('sapn', {'class' : 'u_cbox_count'})
#texts = soup2.find_all('span')
#for i in texts:
#    for j in i.text.split():
#        text.append(j.strip('''"','''))
#text = ' '.join(text)
#keyword = keyword_serch(text)
#text.replace('"', "'")
print(comment_cnt)

None


In [8]:
import pymysql


create table headlines
	(
	ord_id varchar(30) not null,
    title varchar(300) not null,
    press varchar(100) not null,
    categori varchar(20) not null,
    url varchar(100) not null
    );
    
create table detail_news
	(
    ord_id varchar(30) not null,
    reporter varchar(20) not null,
    date int unsigned,
    comment int unsigned,
    text longtext
    );

SyntaxError: invalid syntax (<ipython-input-8-c7bbf29613d3>, line 4)